In [3]:
import requests
from bs4 import BeautifulSoup
import json

### Import list of names from json file:

In [5]:
CharacterNames = []
with open('characters.json') as json_file:
    data = json.load(json_file)
    for p in data['characters']:
        CharacterNames.append('characterName')

### Get webpages

In [7]:
base_url = 'https://gameofthrones.fandom.com/wiki/Stormborn'
index = requests.get(base_url).text
soup_index = BeautifulSoup(index, 'html.parser')
# Find the links on the page
dumps = [a['href'] for a in soup_index.find_all('a') if 
         a.has_attr('href')]

### Get documents

In [8]:
url = 'https://gameofthrones.fandom.com/wiki/Stormborn'
res = requests.get(url)
html_page = res.content
soup = BeautifulSoup(html_page, 'html.parser')
text = soup.find_all(text=True)

output = ''
blacklist = [
    '[document]',
    'noscript',
    'header',
    'html',
    'meta',
    'head', 
    'input',
    'script',
    # there may be more elements you don't want, such as "style", etc.
]

for t in text:
    if t.parent.name not in blacklist:
        output += '{} '.format(t)

print(output)

Stormborn | Game of Thrones Wiki | Fandom 
 
 
 
 
   
 
 
	Games 
 
	Movies 
 
	TV 
 
	Video 
 
 
 Wikis 
   
 
 
 
 
	Explore Wikis 
 
 
 
	Community Central 
 
 
 
	Start a Wiki 
 
 
 
 
 
 
 
 
 
 
 
     
				Search			 
 
 
 
 
 
							This wiki						 
 
 
 
 
 
 
 
 
								This wiki							 
 
 
 
								All wikis							 
 
 
 
				 | 
			 
 
 
 
 
 
   
 
   
 
 
 
 
 
 
 
   
   
 
 
 
 
					Sign In				 
 
 
 Don't have an account? 
 
					Register				 
 
 
 
 
 
 
	Start a Wiki 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 Game of Thrones Wiki 
 
 
 4,654 
 Pages 
 
 
 
   Add new page 
 
 
 
 
 
 
 
 
 Game of Thrones 
 
   
 
 
 
 
 Production 
   
 
 
 
 A Song of Ice and Fire novel series 
 
 
 Game of Thrones television series 
 
 
 Differences between books & TV series 
 
 
 Title sequence 
 
 
 Filming locations 
 
 
 Pronunciation guide 
 
 
 Unaired pilot episode 
 
 
 Costumes 
 
 
 Merchandise 
 
 
 List of references to Game of Thrones in other media 
 
 
 
 
 
 
 Seas